In [24]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.compose import make_column_selector
from sklearn.ensemble import RandomForestClassifier



In [2]:
dataset = pd.read_csv('raw_data/salary.csv')

In [3]:
df = dataset.copy()

In [4]:
df.dtypes

age                int64
workclass         object
fnlwgt             int64
education         object
education-num      int64
marital-status    object
occupation        object
relationship      object
race              object
sex               object
capital-gain       int64
capital-loss       int64
hours-per-week     int64
native-country    object
salary            object
dtype: object

In [5]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [6]:
dropped_columns = ['fnlwgt', 'education-num', 'capital-gain', 'capital-loss']

In [7]:
df.drop(columns= dropped_columns)

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K
32557,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K
32558,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K
32559,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K


In [8]:
salary = {' <=50K': 1,' >50K': 0}
df['salary'] = [salary[item] for item in df['salary']]

In [9]:
df.salary.value_counts()

1    24720
0     7841
Name: salary, dtype: int64

In [10]:
X = df.drop('salary', axis=1)
y = df['salary']

In [11]:
preprocessor = ColumnTransformer([
    ('num_encoder', StandardScaler(), make_column_selector(dtype_include="int64")),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False), make_column_selector(dtype_include="object"))
    ],remainder='passthrough')


preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('num_encoder', StandardScaler(),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x120be0430>),
                                ('cat_encoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x120be04c0>)])

In [12]:
X = pd.DataFrame(preprocessor.fit_transform(X))

/Users/kelvinpurdom/.pyenv/versions/3.9.9/envs/SalaryPrediction/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:828: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
param_grid = {'criterion': ['gini', 'entropy', 'log_loss'],
              'max_features': ['auto', 'sqrt'],
              'max_depth': [2, 10, 20, 30],
              'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72, 80],
              }

In [27]:
rf = RandomForestClassifier()

# Use RandomizedSearchCV to tune the hyperparameters
random_search = RandomizedSearchCV(rf, param_grid, cv=5, n_iter=50, random_state=42)
random_search.fit(X_train, y_train)

/Users/kelvinpurdom/.pyenv/versions/3.9.9/envs/SalaryPrediction/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/kelvinpurdom/.pyenv/versions/3.9.9/envs/SalaryPrediction/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/Users/kelvinpurdom/.pyenv/versions/3.9.9/envs/SalaryPrediction/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and wil

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=50,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [2, 10, 20, 30],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [10, 17, 25, 33, 41, 48,
                                                         56, 64, 72, 80]},
                   random_state=42)

In [31]:
# Print the best hyperparameters
random_search.best_params_

{'n_estimators': 64,
 'max_features': 'auto',
 'max_depth': 20,
 'criterion': 'entropy'}

In [29]:
y_pred = random_search.predict(X_test)

In [30]:
print(accuracy_score(y_test, y_pred))

0.8653462306156917
